## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:44,  7.94it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:39<00:00, 190.82it/s]


In [10]:
tweets_df.head()

user_name     user_location  \
0  FinancialXpress  New Delhi, India   
0          Service                Uk   
0     Fluffy House                     
0     covidnews.ch       Switzerland   
0    Sarah J Black                     

                                    user_description        user_created  \
0  A complete up-to-date source for biz news, #fi... 2009-06-24 11:29:19   
0  Listening and still active for a better place.... 2012-05-16 08:29:37   
0                  Web Shop: https://t.co/2pvY2iKhWv 2014-05-08 15:25:34   
0         Coronavirus Statistics, Information & News 2020-03-21 16:13:52   
0                   wife, mother, educator, advocate 2015-06-19 12:00:57   

   user_followers  user_friends  user_favourites  user_verified  \
0          660388            37               98           True   
0            1475          1872            47807          False   
0             597            57              243          False   
0             353          1882              148          False   
0              89           282             7065          False   

                 date                                               text  \
0 2020-07-28 06:40:35  #Odisha’s #COVID19 tally breached the 28,000 m...   
0 2020-07-28 06:40:35  Media challenges in #COVID19 @BBCBreakfast @GM...   
0 2020-07-28 06:40:34  Listen to Mr. White Cloud 💬\nStay Home, Stay S...   
0 2020-07-28 06:40:33  #CoronaInfoCH #COVID19 #corona #us \nAs COVID-...   
0 2020-07-28 06:40:31  ⁦@WSHHF_Malawi⁩ ⁦@FriendsMalawi⁩ be grateful f...   

                                     hashtags              source  is_retweet  
0                           [Odisha, COVID19]     Twitter Web App       False  
0                                   [COVID19]    Twitter for iPad       False  
0  [fluffyhouse, フラッフィーハウス, 플러피하우스, stayhome]     Twitter Web App       False  
0         [CoronaInfoCH, COVID19, corona, us]           FS Poster       False  
0                                        None  Twitter for iPhone       False

## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (32176, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 32176 all tweets: 39676


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (39676, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)